In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

def plot_confusion_matrix(cm, class_names, output_matrix_path):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.savefig(output_matrix_path)
    plt.close()

def evaluate_model(model, generator, output_matrix_path):
    predictions = model.predict(generator)
    preds = np.argmax(predictions, axis=1)
    labels = generator.classes
    class_names = list(generator.class_indices.keys())

    cm = confusion_matrix(labels, preds)
    plot_confusion_matrix(cm, class_names, output_matrix_path)

    report = classification_report(labels, preds, target_names=class_names)
    return report, cm

def plot_training_history(history, output_dir):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    history_plot_path = os.path.join(output_dir, "training_history.png")
    plt.savefig(history_plot_path)
    plt.close()

if __name__ == '__main__':
    output_dir = "outputarqum"
    os.makedirs(output_dir, exist_ok=True)

    model_save_path = os.path.join(output_dir, "cnn_model.keras")
    train_conf_matrix_path = os.path.join(output_dir, "train_confusion_matrix.png")
    val_conf_matrix_path = os.path.join(output_dir, "val_confusion_matrix.png")
    test_conf_matrix_path = os.path.join(output_dir, "test_confusion_matrix.png")
    classification_report_path = os.path.join(output_dir, "classification_report.txt")

    train_dir = r"E:\AI TensorOpenEndedLab\AI Open\CT-Scan\CT-Scan Train"
    test_dir = r"E:\AI TensorOpenEndedLab\AI Open\CT-Scan\CT-Scan Test"

    train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range=30,
        horizontal_flip=True,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        validation_split=0.2
    )

    test_datagen = ImageDataGenerator(rescale=1.0/255.0)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical',
        subset='training'
    )

    val_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical',
        subset='validation'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical',
        shuffle=False
    )

    base_model = tf.keras.applications.ResNet50(
        weights='imagenet', include_top=False, input_shape=(224, 224, 3)
    )
    base_model.trainable = False  

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(train_generator.num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=8
    )

    model.save(model_save_path)
    print(f"Model saved at {model_save_path}")

    plot_training_history(history, output_dir)

    train_report, train_cm = evaluate_model(model, train_generator, train_conf_matrix_path)
    val_report, val_cm = evaluate_model(model, val_generator, val_conf_matrix_path)
    test_report, test_cm = evaluate_model(model, test_generator, test_conf_matrix_path)

    with open(classification_report_path, "w") as f:
        f.write("Classification Report - Training:\n")
        f.write(train_report + "\n")
        f.write("\nClassification Report - Validation:\n")
        f.write(val_report + "\n")
        f.write("\nClassification Report - Testing:\n")
        f.write(test_report)

    print("Classification report saved!")


Found 2888 images belonging to 4 classes.
Found 720 images belonging to 4 classes.
Found 999 images belonging to 4 classes.


C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 615s 3s/step - accuracy: 0.3241 - loss: 1.5037 - val_accuracy: 0.3653 - val_loss: 1.3572
Epoch 2/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 585s 3s/step - accuracy: 0.4743 - loss: 1.2116 - val_accuracy: 0.5028 - val_loss: 1.1590
Epoch 3/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 560s 3s/step - accuracy: 0.5451 - loss: 1.0579 - val_accuracy: 0.4958 - val_loss: 1.0929
Epoch 4/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 567s 3s/step - accuracy: 0.5781 - loss: 0.9948 - val_accuracy: 0.4889 - val_loss: 1.1110
Epoch 5/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 618s 3s/step - accuracy: 0.6074 - loss: 0.9389 - val_accuracy: 0.5542 - val_loss: 1.0532
Epoch 6/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 565s 3s/step - accuracy: 0.6483 - loss: 0.8713 - val_accuracy: 0.5486 - val_loss: 1.0440
Epoch 7/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - accuracy: 0.6312 - loss: 0.8815 - val_accuracy: 0.5583 - val_loss: 1.0596
Epoch 8/8
181/181 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - accuracy: 0.6907 - loss: 0.7989 - val_accuracy: 0.

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 170s 3s/step
Classification report saved!
